In [1]:
#!pip install minsearch

In [32]:
import minsearch, json

In [33]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [34]:
#docs_raw

In [35]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [36]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [37]:
index = minsearch.Index(
    text_fields=['question','section','text'],
    keyword_fields=['course']
)

In [38]:
index.fit(documents)

In [39]:
q = 'course already started, can i still enroll'

In [40]:
# boost = {'question':3.0, 'text':0.5}

# results = index.search(
#     query=q,
#     boost_dict=boost,
#     num_results = 10
# )

In [41]:
from google import genai
import os
os.environ["GOOGLE_API_KEY"] =  "YOUR API KEY HERE"

In [42]:
client = genai.Client()

response = client.models.generate_content(
            model="gemini-2.0-flash", 
            contents="Explain how AI works in a few words?"
        )
print(response.text)

AI works by learning patterns from data to make predictions or decisions.



In [43]:
def search(query):
    boost = {'question':3.0, 'text':0.5}
    
    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results = 10
        )
    return results
    

In [44]:
def build_prompt(query, search_results):
    prompt_template = """
        You are a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT from the FAQ document
        Use only the facts from the CONTEXT while answering the QUESTION
        If the CONTEXT does not contain the answer, output NONE
        QUESTION: {question}
        CONTEXT: {context}
        
        """.strip()
    context =""
    for doc in search_results:
        context = context + f"section: {doc['section']}\n question: {doc['question']}\n answer: {doc['text']}\n\n " 
    #print(context)
    prompt = prompt_template.format(question = query, context =context).strip()
    #print(query)
    return prompt

In [45]:
def llm(prompt):
    #lets do it in gemini
    response = client.models.generate_content(
                model="gemini-2.0-flash", 
                contents= prompt 
            )
    return response.text

In [46]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return(answer)

In [47]:
rag('course already started, can i still enroll?')

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [48]:
rag("how do I run kafka")

'```text\nIn the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```'

In [49]:
rag('How do i enroll in the course')

'Register before the course starts using this link.'